In [1]:
!pip install xmltodict
!pip install nltk
!python -m nltk.downloader all

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /roo

In [25]:
# df_click = pd.read_csv('Datasets/clickstream-enwiki-2020-01.tsv', delimiter='\t', encoding='utf-8', names=['referer', 'resource', 'path', 'count'])
import pandas as pd
import xmltodict, nltk
import json
import re, string, ast
import numpy as np

from gensim import models
from gensim.corpora import Dictionary, MmCorpus
from nltk import download, tokenize, word_tokenize, pos_tag 
from nltk.corpus import stopwords
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
#df_click.head(10)

# New Section

In [0]:
# with open('Datasets/enwiki-20200101-pages-articles-multistream-index1.txt-p10p30302', encoding='utf8') as file:
#     data_index = file.read()

In [0]:
with open('enwiki-20200101-pages-articles-multistream1.xml-p10p30302', encoding='utf8') as file:
    #data_text = file.read()
    doc = xmltodict.parse(file.read())

In [0]:
# print(doc['mediawiki']['page'][11])

In [0]:
df_text = pd.DataFrame(columns=['title', 'text', 'wiki_link', 'redirect'])

for page in doc['mediawiki']['page']:
    title = page['title']
    text_w = ''
    text_s = ''
    wiki_link = ''
    redirect = 'F'
    
    if 'redirect' in page:
        # only keeping redirecting link
        txt = re.search('(\[\[(.*?)\]\])', page['revision']['text']['#text']).group(1)
        txt = re.sub('\[*\]*', '', txt)
        redirect = 'T'
        wiki_link = txt.strip()
        
    else:
        # getting rid of {{~}}, [[File:~]], <!-- ~ -->, <ref ~ />, <ref ~</ref>, <br~>
        txt = re.sub(r'({{(.*?)}})|(\[\[File:(.*?)\n)|(\<\!\-\-(.*?)\-\-\>)|(\<ref(.*?)\/\>)|(\<ref(.*?)\<\/ref\>)|(\<br(\s?\/?)\>)', 
                     '', page['revision']['text']['#text'], 0, re.DOTALL)
        
        # separating internal links
        link = re.findall('(\[\[(.*?)\]\])', txt)
        text_w = re.sub('(\[\[(.*?)\]\])|(\\n)', ' ', txt, 0, re.DOTALL)
        text_s = re.sub('(?<=^\[\[\b).*(?=\b\|(.*?)\]\])|(\\n)',' ',txt, 0, re.DOTALL) #keeping the links first
        text_s = re.sub('(\[\[(.*?)\]\])|(\\n)', ' ', text_s, 0, re.DOTALL)
        
        for c in link:
            if '|' in c[1]:
                sep = c[1].split('|')
                wiki_link = wiki_link + ', ' + sep[0]
                text_w = text_w + ', ' + sep[1]
            else:
                wiki_link = wiki_link + ', ' + c[1]
                text_w = text_w + ', ' + c[1]
                
    df_text = df_text.append({'title': title, 'text': text_w, 'wiki_link': wiki_link, 'redirect': redirect, 'sentences': text_s}, ignore_index=True) 
    

In [0]:
# df_text['sentences'][1] # full article text for Anarchism article

In [0]:
def preprocess_sentence(doc):
    return tokenize.sent_tokenize(doc)

In [0]:
df_text['sentences'] = df_text['sentences'].apply(preprocess_sentence)

In [11]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def preprocess_word(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    while (doc.count('n')): 
        doc.remove('n') 
    while (doc.count('br')): 
        doc.remove('br') 
    return doc

In [0]:
df_text['text'] = df_text['text'].apply(preprocess_word)

In [0]:
def preprocess_link(doc):
    if doc.startswith(', '):
        doc = doc[2:]
    doc = doc.split(', ')
    return doc

In [0]:
df_text['wiki_link'] = df_text['wiki_link'].apply(preprocess_link)

# Text Modelling for each topic

In [16]:
df_text.head(5)

,title,text,wiki_link,redirect,sentences
0,AccessibleComputing,[],[Computer accessibility],T,[]
1,Anarchism,"[anarchism, rejects, deemed, unjust, advocates...","[Anti-authoritarianism, Political philosophy, ...",F,[ '''Anarchism''' is an and that ...
2,AfghanistanHistory,[],[History of Afghanistan],T,[]
3,AfghanistanGeography,[],[Geography of Afghanistan],T,[]
4,AfghanistanPeople,[],[Demographics of Afghanistan],T,[]


In [0]:
def filter_words(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2] == 'RB'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [18]:
#Picks out nouns, adverbs, adjectives and removes unwanted characters of each sentence for each article
final_ls = []
for i in range(0, len(df_text)):
    fil_sent = df_text['sentences'][i]
    sen_list = []
    for j in range(0,len(fil_sent)):
        b = filter_words(fil_sent[j])
        res = re.sub('['+string.punctuation+']', '', b).split() 
        listToStr = ' '.join([str(val) for val in res]) 
        sen_list.append(listToStr)
    listToStr2 = ' '.join([str(val) for val in sen_list])
    final_ls.append(listToStr2)
new_df = pd.DataFrame({'tokens': final_ls})
new_df

,tokens
0,
1,Anarchism unjust replacement societies volunta...
2,
3,
4,
...,...
19810,
19811,Tax Freedom Day first day year whole theoretic...
19812,tax compulsory financial charge other type lev...
19813,Transhumanism H h international transformation...


In [19]:
df_text['tokens'] = new_df['tokens'].apply(word_tokenize)
df_text

,title,text,wiki_link,redirect,sentences,tokens
0,AccessibleComputing,[],[Computer accessibility],T,[],[]
1,Anarchism,"[anarchism, rejects, deemed, unjust, advocates...","[Anti-authoritarianism, Political philosophy, ...",F,[ '''Anarchism''' is an and that ...,"[Anarchism, unjust, replacement, societies, vo..."
2,AfghanistanHistory,[],[History of Afghanistan],T,[],[]
3,AfghanistanGeography,[],[Geography of Afghanistan],T,[],[]
4,AfghanistanPeople,[],[Demographics of Afghanistan],T,[],[]
...,...,...,...,...,...,...
19810,The Lord of the Rings/One Ring,[],[One Ring],T,[],[]
19811,Tax Freedom Day,"[tax, freedom, day, first, day, year, whole, t...","[nation, Tax Foundation, government, Governmen...",F,['''Tax Freedom Day''' is the first day of the...,"[Tax, Freedom, Day, first, day, year, whole, t..."
19812,Tax,"[tax, compulsory, financial, charge, type, lev...","[Latin, wikt:en:taxo#Latin, Legal person, tax ...",F,[ A '''tax''' (from the '' '') is a compul...,"[tax, compulsory, financial, charge, other, ty..."
19813,Transhumanism,"[transhumanism, abbreviated, international, ad...","[school of thought, human condition, human enh...",F,[ '''Transhumanism''' (abbreviated as '''...,"[Transhumanism, H, h, international, transform..."


In [0]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [0]:
def model_all():
    dictionary, corpus = prep_corpus(df_text['tokens'])
    MmCorpus.serialize('wiki_articles.mm', corpus)
    dictionary.save('wiki_articles.dict')
    lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=100)
    lda.save('anarchism.model')
    lda.show_topics(formatted=False)

In [0]:
def model_single():
    single_topic_list = []
    for i in df_text['tokens']:
        dictionary, corpus = prep_corpus(df_text['tokens'])
        MmCorpus.serialize('wiki_articles.mm', corpus)
        dictionary.save('wiki_articles.dict')
        lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, passes=50)
        lda.save('anarchism.model')
        single_topic_list.append(lda.show_topics(formatted=False))


In [0]:
model_single()
print(single_topic_list)

Building dictionary...
Building corpus...


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Building dictionary...
Building corpus...
Building dictionary...
Building corpus...
Building dictionary...
Building corpus...
Building dictionary...
Building corpus...


In [0]:
# MmCorpus.serialize('wiki_articles.mm', corpus)
# dictionary.save('wiki_articles.dict')

In [0]:
# %%time
# lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=100)
                                      
# lda.save('anarchism.model')

In [0]:
# lda.show_topics(formatted=False)
# print("")